In [2]:
import numpy as np
from keras import layers
from keras.layers import Input, Add, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D, AveragePooling2D, MaxPooling2D, GlobalAveragePooling2D,GlobalMaxPooling2D
from keras.models import Model, load_model
from keras.preprocessing import image
from keras.utils import layer_utils
from keras.utils.data_utils import get_file
from keras.applications.imagenet_utils import preprocess_input
import pydot
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot
from keras.utils import plot_model
from keras.initializers import glorot_uniform
import scipy.misc
from matplotlib.pyplot import imshow
%matplotlib inline

import keras.backend as K
K.set_image_data_format('channels_last')
K.set_learning_phase(1)
import pandas as pd

/home/johndanielmathew/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
/home/johndanielmathew/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [3]:
import os
import sys
import numpy as np
import pandas as pd
from keras.preprocessing import image
from sklearn.preprocessing import LabelEncoder
from keras.utils.np_utils import to_categorical

In [4]:
data_root = "../leaf/"
img_folder = data_root + 'images/'
train_data = pd.read_csv(data_root + 'train.csv')
train_ID = train_data['id']
train_Y = train_data['species']
test_data = pd.read_csv(data_root + 'test.csv')
test_ID = test_data['id']

le = LabelEncoder()
train_y = le.fit_transform(train_Y)





## Preprocessing the images
 1.)Load the images
 2.)Normalize

In [5]:
def resize_img(img, max_dim=96):
    large_axis = max((0, 1), key=lambda x: img.size[x])
    scalar = max_dim / float(img.size[large_axis])
    resized = img.resize(
        (int(img.size[0] * scalar), int(img.size[1] * scalar)))
    return resized


def load_image_data(id_list, max_dim=96, c = 1, center=True):
    X = np.empty((len(id_list), max_dim, max_dim, c)
    for i, idnum in enumerate(id_list):
        x = image.load_img(
            (img_folder + str(idnum) + '.jpg'), grayscale=False)
        x = image.img_to_array(resize_img(x, max_dim=max_dim))
        height = x.shape[0]
        width = x.shape[1]
        if center:
            h1 = int((max_dim - height) / 2)
            h2 = h1 + height
            w1 = int((max_dim - width) / 2)
            w2 = w1 + width
        else:
            h1, w1 = 0, 0
            h2, w2 = (height, width)
        X[i, h1:h2, w1:w2, :] = x
    return np.around(X / 255.0)



In [6]:
# Tweaked the max_dim of load_image_data() 
# Resnet implemtation max_dim = 96 and c = 1
# For finetuning its 224 and 3 respectively

train_X = load_image_data(train_ID,224)
train_y = to_categorical(train_y,99)
X_test = load_image_data(test_ID,224)
print(train_X.shape,train_y.shape)


(990, 224, 224, 3) (990, 99)


# Resnet50 - Implementation
## identity block

In [20]:
# identity_block

def identity_block(X, f, filters, stage, block):
    
    # defining name basis
    conv_name_base = 'res' + str(stage) + block + '_branch'
    bn_name_base = 'bn' + str(stage) + block + '_branch'
    
    F1, F2, F3 = filters
    
    X_shortcut = X
    X = Conv2D(filters = F1, kernel_size = (1, 1), strides = (1,1), padding = 'valid', name = conv_name_base + '2a', kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3, name = bn_name_base + '2a')(X)
    X = Activation('relu')(X)

    X = Conv2D(filters = F2, kernel_size = (f, f), strides = (1,1), padding = 'same', name = conv_name_base + '2b', kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3, name = bn_name_base + '2b')(X)
    X = Activation('relu')(X)

    X = Conv2D(filters = F3, kernel_size = (1, 1), strides = (1,1), padding = 'valid', name = conv_name_base + '2c', kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3, name = bn_name_base + '2c')(X)

    X = Add()([X_shortcut , X])
    X = Activation('relu')(X)

    return X

In [21]:

def convolutional_block(X, f, filters, stage, block, s = 2):
    conv_name_base = 'res' + str(stage) + block + '_branch'
    bn_name_base = 'bn' + str(stage) + block + '_branch'
    
    F1, F2, F3 = filters
    
    X_shortcut = X


    X = Conv2D(F1, (1, 1), strides = (s,s),padding="valid", name = conv_name_base + '2a', kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3, name = bn_name_base + '2a')(X)
    X = Activation('relu')(X)
    
    X = Conv2D(F2, (f, f), strides = (1,1),padding="same", name = conv_name_base + '2b', kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3, name = bn_name_base + '2b')(X)
    X = Activation('relu')(X)

    X = Conv2D(F3, (1, 1), strides = (1,1),padding="valid", name = conv_name_base + '2c', kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3, name = bn_name_base + '2c')(X)

    X_shortcut = Conv2D(F3, (1, 1), strides = (s,s),padding="valid", name = conv_name_base + '1', kernel_initializer = glorot_uniform(seed=0))(X_shortcut)
    X_shortcut = BatchNormalization(axis = 3, name = bn_name_base + '1')(X_shortcut)

    X = Add()([X , X_shortcut])
    X = Activation('relu')(X)
    
    
    return X

In [22]:
#  ResNet50

def ResNet50(input_shape = (96, 96, 1), classes = 6):

    
    # Define the input as a tensor with shape input_shape
    X_input = Input(input_shape)

    
    # Zero-Padding
    X = ZeroPadding2D((3, 3))(X_input)
    
    # Stage 1
    X = Conv2D(64, (7, 7), strides = (2, 2), name = 'conv1', kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3, name = 'bn_conv1')(X)
    X = Activation('relu')(X)
    X = MaxPooling2D((3, 3), strides=(2, 2))(X)

    # Stage 2
    X = convolutional_block(X, f = 3, filters = [64, 64, 256], stage = 2, block='a', s = 1)
    X = identity_block(X, 3, [64, 64, 256], stage=2, block='b')
    X = identity_block(X, 3, [64, 64, 256], stage=2, block='c')


    # Stage 3 
    X = convolutional_block(X, f = 3, filters = [128, 128, 512], stage = 3, block='a', s = 2)
    X = identity_block(X, 3, [128, 128, 512], stage=3, block='b')
    X = identity_block(X, 3, [128, 128, 512], stage=3, block='c')
    X = identity_block(X, 3, [128, 128, 512], stage=3, block='d')

    # Stage 4 
    X = convolutional_block(X, f = 3, filters =  [256, 256, 1024], stage = 4, block='a', s = 2)
    X = identity_block(X, 3, [256, 256, 1024], stage=4, block='b')
    X = identity_block(X, 3, [256, 256, 1024], stage=4, block='c')
    X = identity_block(X, 3, [256, 256, 1024], stage=4, block='d')
    X = identity_block(X, 3, [256, 256, 1024], stage=4, block='e')
    X = identity_block(X, 3, [256, 256, 1024], stage=4, block='f')
    
    # Stage 5 
    X = convolutional_block(X, f = 3, filters =  [512, 512, 2048], stage = 5, block='a', s = 2)
    X = identity_block(X, 3, [512, 512, 2048], stage=5, block='b')
    X = identity_block(X, 3, [512, 512, 2048], stage=5, block='c')

    X = AveragePooling2D(pool_size=(2, 2), name = "avg_pool")(X)
    

    X = Flatten()(X)
    X = Dense(classes, activation='softmax', name='fc' + str(classes), kernel_initializer = glorot_uniform(seed=0))(X)
    
    
    model = Model(inputs = X_input, outputs = X, name='ResNet50')

    return model

In [23]:
model = ResNet50(input_shape = (96, 96, 1), classes = 99)

In [10]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [24]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_4 (InputLayer)            (None, 96, 96, 1)    0                                            
__________________________________________________________________________________________________
zero_padding2d_1 (ZeroPadding2D (None, 102, 102, 1)  0           input_4[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 48, 48, 64)   3200        zero_padding2d_1[0][0]           
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 48, 48, 64)   256         conv1[0][0]                      
__________________________________________________________________________________________________
activation

In [15]:
model.load_weights("./model2.h5")

In [16]:
model.fit(train_X, train_y, epochs = 20, batch_size = 64)

Epoch 1/20
990/990 [==============================] - 110s 111ms/step - loss: 0.0129 - acc: 0.9960
Epoch 2/20
990/990 [==============================] - 111s 112ms/step - loss: 0.0050 - acc: 0.9990
Epoch 3/20
990/990 [==============================] - 110s 112ms/step - loss: 0.0038 - acc: 0.9990
Epoch 4/20
990/990 [==============================] - 111s 112ms/step - loss: 0.0035 - acc: 0.9990
Epoch 5/20
990/990 [==============================] - 111s 112ms/step - loss: 0.0039 - acc: 0.9980
Epoch 6/20
990/990 [==============================] - 111s 112ms/step - loss: 0.0048 - acc: 0.9990
Epoch 7/20
990/990 [==============================] - 111s 112ms/step - loss: 0.0026 - acc: 0.9990
Epoch 8/20
990/990 [==============================] - 111s 112ms/step - loss: 0.0143 - acc: 0.9980
Epoch 9/20
990/990 [==============================] - 112s 113ms/step - loss: 0.0209 - acc: 0.9939
Epoch 10/20
990/990 [==============================] - 111s 113ms/step - loss: 0.0139 - acc: 0.9949
Epoch 11/

In [18]:
model.save_weights("./model2_95.h5")

In [17]:
X_test = load_image_data(test_ID)
Resnet_pred = model.predict(X_test)
LABELS = sorted(pd.read_csv(os.path.join(data_root, 'train.csv')).species.unique())
save_File = pd.DataFrame(Resnet_pred,index=test_ID,columns=LABELS)
save_File.to_csv("submission.csv", index=True)

## Transfer Learning - Fine-tuning Resnet50 model trained on imagenet
The 900 images is less. Finetuning a resnet50 model will a good option. I tried VGG16 but could not train it due to limited Hardware(No GPUs). Tried both Adam and sgd optimizers and adam is giving better results and manually decayed learning rate. 

In [7]:
import keras
from keras.models import Sequential
from keras.optimizers import Adam,SGD

In [8]:
resnet50_model = keras.applications.resnet50.ResNet50(include_top=False)

In [9]:
resnet50_model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, None, None, 3 0                                            
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, None, None, 6 9472        input_1[0][0]                    
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, None, None, 6 256         conv1[0][0]                      
__________________________________________________________________________________________________
activation_1 (Activation)       (None, None, None, 6 0           bn_conv1[0][0]                   
__________________________________________________________________________________________________
max_poolin

In [10]:
X = resnet50_model.output
X = GlobalAveragePooling2D()(X)
X = Dense(512, activation='relu')(X)
predictions = Dense(99, activation='softmax')(X)

In [11]:
model2 = Model(inputs=resnet50_model.input, outputs=predictions)

### Just made only 0.1 percent of the layers trainable because of limitation of hardware.  

In [12]:
layer_num = len(model2.layers)
for layer in model2.layers[:int(layer_num * 0.9)]:
    layer.trainable = False
for layer in model2.layers[int(layer_num * 0.9):]:
        layer.trainable = True

In [30]:
model2.compile(optimizer=Adam(lr=0.00001), loss='categorical_crossentropy', metrics=['accuracy'])

In [24]:
model2.load_weights("./model2_1.h5")

In [31]:
model2.fit(train_X, train_y, epochs = 20, batch_size = 64)

Epoch 1/20
990/990 [==============================] - 226s 228ms/step - loss: 6.1654e-05 - acc: 1.0000
Epoch 2/20
990/990 [==============================] - 225s 228ms/step - loss: 4.2828e-05 - acc: 1.0000
Epoch 3/20
990/990 [==============================] - 226s 228ms/step - loss: 3.6823e-05 - acc: 1.0000
Epoch 4/20
990/990 [==============================] - 226s 228ms/step - loss: 3.6591e-05 - acc: 1.0000
Epoch 5/20
990/990 [==============================] - 226s 228ms/step - loss: 3.2909e-05 - acc: 1.0000
Epoch 6/20
990/990 [==============================] - 226s 228ms/step - loss: 4.9727e-05 - acc: 1.0000
Epoch 7/20
990/990 [==============================] - 226s 228ms/step - loss: 3.9903e-05 - acc: 1.0000
Epoch 8/20
990/990 [==============================] - 226s 228ms/step - loss: 3.2135e-05 - acc: 1.0000
Epoch 9/20
990/990 [==============================] - 226s 228ms/step - loss: 2.7753e-05 - acc: 1.0000
Epoch 10/20
990/990 [==============================] - 226s 228ms/step - 

In [26]:
model2.save_weights("./model2_13.h5")

In [32]:

Resnet_pred = model2.predict(X_test)
LABELS = sorted(pd.read_csv(os.path.join(data_root, 'train.csv')).species.unique())
save_File = pd.DataFrame(Resnet_pred,index=test_ID,columns=LABELS)
save_File.to_csv("submission.csv", index=True)